In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.patches as mpatches

In [ ]:
import os
print(os.getcwd())

In [ ]:
# in this section, write the script to load the data and complete the main part of the analysis.
# try to print the results to the screen using the format method demonstrated in the workbook

# load the necessary data here and transform to a UTM projection
counties = gpd.read_file('data_files/Counties.shp')
wards = gpd.read_file('data_files/NI_Wards.shp')
counties = counties.to_crs(epsg=32629)
wards = wards.to_crs(epsg=32629)

# your analysis goes here...
join = gpd.sjoin(counties, wards, how = 'inner', lsuffix = 'left', rsuffix = 'right')
join.head()

In [ ]:
county_total_popn = join.groupby('CountyName')['Population'].sum()
print(county_total_popn)

In [ ]:
highest_popn_county = county_total_popn.idxmax()
county_popn_max = county_total_popn.max()
print(f'The county with the highest population is {highest_popn_county}, with a population of {county_popn_max}')

In [ ]:
lowest_popn_county = county_total_popn.idxmin()
county_popn_min = county_total_popn.min()
print(f'The county with the lowest population is {lowest_popn_county}, with a population of {county_popn_min}')

In [ ]:
ward_counties = join.groupby('Ward')['CountyName'].nunique() # group by ward and count number of unique counties associated with
wards_multi_county = ward_counties[ward_counties > 1] # filters wards associated with more than 1 county
num_wards_multi_county = len(wards_multi_county) # counts number of wards located in more than 1 county
wards_multi = join[join['Ward'].isin(wards_multi_county.index)] # get total population of wards
popn_multi_county = wards_multi['Population'].sum()
ward_popn_max = join.loc[join['Population'].idxmax()]
# ward_popn_min = join.loc[join['Population'].idxmin()]

print(f'Wards in multiple counties are: {wards_multi_county}')
print(f'The number of wards located in more than one county is {num_wards_multi_county}')
print(wards_multi)
print(f'Ward with the highest population: {ward_popn_max['Ward']} with a population of {ward_popn_max['Population']}')
# print(f'Ward with the lowest population: {ward_popn_min['Ward']} with a population of {ward_popn_min['Population']}')

In [ ]:
wards.head()

In [ ]:
wards['Area_SqKm'] = wards.geometry.area / 1000000 # create area column from m2 to km2
wards.head()

In [ ]:
join.head()

In [ ]:
# repeated using population density instead of population
wards['Population_Density'] = wards['Population'] / wards['Area_SqKm']

# Find the Ward with the highest population density
ward_max_density = wards.loc[wards['Population_Density'].idxmax()]

# Find the Ward with the lowest population density
ward_min_density = wards.loc[wards['Population_Density'].idxmin()]

print(f"Ward with the highest population density: {ward_max_density['Ward']} with a population density of {ward_max_density['Population_Density']} people per SqKM")
print(f"Ward with the lowest population density: {ward_min_density['Ward']} with a population density of {ward_min_density['Population_Density']} people per SqKM")
wards.head()

In [ ]:
wards['Population_Density'].hist(bins=10, edgecolor='black')
plt.show()

In [ ]:
import numpy as np 
wards['log_pop_density'] = np.log(wards['Population_Density'])
wards['log_pop_density'].hist(bins=10, edgecolor='black')
plt.show()
wards.head()

In [ ]:
popn_dens_max = wards['log_pop_density'].max()
popn_dens_min = wards['log_pop_density'].min()
print(popn_dens_max)
print(popn_dens_min)

In [ ]:
def generate_handles(labels, colors, edge='k', alpha=1):
    """
    Generate matplotlib patch handles to create a legend of each of the features in the map.

    Parameters
    ----------

    labels : list(str)
        the text labels of the features to add to the legend

    colors : list(matplotlib color)
        the colors used for each of the features included in the map.

    edge : matplotlib color (default: 'k')
        the color to use for the edge of the legend patches.

    alpha : float (default: 1.0)
        the alpha value to use for the legend patches.

    Returns
    -------

    handles : list(matplotlib.patches.Rectangle)
        the list of legend patches to pass to ax.legend()
    """
    lc = len(colors)  # get the length of the color list
    handles = [] # create an empty list
    for ii in range(len(labels)): # for each label and color pair that we're given, make an empty box to pass to our legend
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[ii % lc], edgecolor=edge, alpha=alpha))
    return handles

# adapted this question: https://stackoverflow.com/q/32333870
# answered by SO user Siyh: https://stackoverflow.com/a/35705477
def scale_bar(ax, length=20, location=(0.92, 0.95)):
    """
    Create a scale bar in a cartopy GeoAxes.

    Parameters
    ----------

    ax : cartopy.mpl.geoaxes.GeoAxes
        the cartopy GeoAxes to add the scalebar to.
    
    length : int, float (default 20)
        the length of the scalebar, in km
    
    location : tuple(float, float) (default (0.92, 0.95))
        the location of the center right corner of the scalebar, in fractions of the axis.

    Returns
    -------
    ax : cartopy.mpl.geoaxes.GeoAxes
        the cartopy GeoAxes object

    """
    x0, x1, y0, y1 = ax.get_extent() # get the current extent of the axis
    sbx = x0 + (x1 - x0) * location[0] # get the right x coordinate of the scale bar
    sby = y0 + (y1 - y0) * location[1] # get the right y coordinate of the scale bar

    ax.plot([sbx, sbx-length*1000], [sby, sby], color='k', linewidth=4, transform=ax.projection) # plot a thick black line
    ax.plot([sbx-(length/2)*1000, sbx-length*1000], [sby, sby], color='w', linewidth=2, transform=ax.projection) # plot a white line from 0 to halfway

    ax.text(sbx, sby-(length/4)*1000, f"{length} km", ha='center', transform=ax.projection, fontsize=6) # add a label at the right side
    ax.text(sbx-(length/2)*1000, sby-(length/4)*1000, f"{int(length/2)} km", ha='center', transform=ax.projection, fontsize=6) # add a label in the center
    ax.text(sbx-length*1000, sby-(length/4)*1000, '0 km', ha='center', transform=ax.projection, fontsize=6) # add a label at the left side

    return ax

In [ ]:
ni_utm = ccrs.UTM(29)

# create a figure of size 10x10 (representing the page size in inches
fig, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw=dict(projection=ni_utm))

# add gridlines below
gridlines = ax.gridlines(draw_labels = True,
                         xlocs=[-8, -7.5, -7, -6.5, -6, -5.5],
                         ylocs=[54, 54.5, 55, 55.5])
gridlines.right_labels = False
gridlines.bottom_labels = False


# to make a nice colorbar that stays in line with our map, use these lines:
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1, axes_class=plt.Axes)

# plot the ward data into our axis, using gdf.plot()
ward_plot = wards.plot(column='log_pop_density', ax=ax, vmin=0, vmax=10, cmap='viridis',
                       legend=True, cax=cax, legend_kwds={'label': 'Log of Population Density'})

# add ward outlines in red using ShapelyFeature
ward_outlines = ShapelyFeature(wards['geometry'], ni_utm, edgecolor='r', facecolor='none')
ax.add_feature(ward_outlines)

ward_handles = generate_handles([''], ['none'], edge='r')

# add a legend in the upper left-hand corner
ax.legend(ward_handles, ['Ward Boundaries'], fontsize=12, loc='upper left', framealpha=1)

# save the figure
fig.savefig('sample_map.png', dpi=300, bbox_inches='tight')


In [ ]:
ax


In [ ]:
ax.get_extent()